In [3]:
import gym
from cv2 import cv2
import time
import numpy as np
from tqdm import tqdm
import os
import random

In [9]:
# create a random weights array once to be used to bring the 'observation' to 1000 units.
sizeOfArray = 256

if os.path.exists('weights_'+str(sizeOfArray)+'.npy'):
    array_weights_for_observation = np.load('weights_'+str(sizeOfArray)+'.npy')
else:
    array_weights_for_observation = np.random.rand(1,sizeOfArray)
    np.save('weights_'+str(sizeOfArray)+'.npy',array_weights_for_observation)

array_weights_for_observation = array_weights_for_observation.reshape(sizeOfArray)

In [5]:
X = []
X_obs = []
X_img = []
Y = []
initial_games = 200
target_score = 350

In [1]:
env_name = "Assault-ram-v0"

In [17]:
#env_name = "Boxing-ram-v0"
env = gym.make(env_name)
for _ in tqdm(range(initial_games)):
    env.reset()
    score = 0
    prev_obs = []
    X_obs = []
    Y_temp = []
    while True:
        env.render()
        #time.sleep(0.05)
        img = env.render(mode = 'rgb_array')
        img = cv2.resize(img, (100, 100))
        action = env.action_space.sample() # your agent here (this takes random actions)
        observation, reward, done, info = env.step(action)
        
        
        if len(observation) > sizeOfArray:     # get all observation array to size of 1000 before saving
            if len(observation) % sizeOfArray == 0:
                observation = np.array(observation)
                observation = observation.reshape(-1,int(observation.size/sizeOfArray)).mean(axis=1)
                observation = np.convolve(np.array(observation),array_weights_for_observation,'same')
                
            else:
                observation.append([0]*(sizeOfArray-(len(observation)%sizeOfArray)))
                observation = np.array(observation)
                observation = observation.reshape(-1,int(observation.size/sizeOfArray)).mean(axis=1)
                observation = np.convolve(np.array(observation),array_weights_for_observation,'same')
                
        else:
            observation = np.array(observation)
            #print(observation.shape)
            #print(array_weights_for_observation.shape)
            observation = np.convolve(observation,array_weights_for_observation,'same')
            
        #print(action)
        if prev_obs != []:
            temp_Y = [0] * 10
            temp_Y[action] = 1
            Y_temp.append(temp_Y)
            score += reward
            X_obs.append(img)
            X_obs.append(prev_obs)
            #X.append(score)
        prev_obs = list(observation)
        #print(score)
        if done:
            break
    if score >= target_score:
        X.append(X_obs)
        Y.append(Y_temp)




  0%|          | 0/200 [00:00<?, ?it/s]Exception KeyError: KeyError(<weakref at 0x7fbdb70d7fc8; to 'tqdm' at 0x7fbdb0468f50>,) in <bound method tqdm.__del__ of   0%|          | 0/200 [00:00<?, ?it/s]> ignored


ValueError: object too deep for desired array

In [19]:
X[0][1].shape

IndexError: index 0 is out of bounds for axis 0 with size 0

In [6]:
len(X)
X = np.array(X)
Y = np.array(Y)
X_final = []
Y_final = []
for i in tqdm(range(int(X.size))):
    for j in range(len(X[i])):
        X_final.append(X[i][j])
        
    for j in range(len(Y[i])):
        Y_final.append(Y[i][j])

100%|██████████| 22/22 [00:00<00:00, 1587.77it/s]


In [7]:
X = np.array(X_final)
Y = np.array(Y_final)
X_final = []
Y_final = []

In [8]:
X.shape

(14268, 2)

In [8]:
X = X.reshape(int(X.size/2),2)

In [9]:
Y.shape

(19804, 10)

In [10]:
np.save(env_name+'_X_10.npy',X)
np.save(env_name+'_Y_10.npy',Y)

In [6]:
X = np.load(env_name+'_X_10.npy')
Y = np.load(env_name+'_Y_10.npy')

In [10]:
np.save(env_name+'_X.npy',X)
np.save(env_name+'_Y.npy',Y)

In [4]:
X = np.load(env_name+'_X.npy')
Y = np.load(env_name+'_Y.npy')

In [7]:
import tensorflow as tf
import tflearn 
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [12]:

tf.reset_default_graph()
IMG_SIZE = 100
LR = 1e-3

MODEL_NAME = 'common_agent_9'


convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 3], name='input-1')

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 128, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 256, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 128, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = tf.reshape(convnet, shape=[-1, 16, 2])
convnet =  tflearn.lstm(convnet, 1024, activation='relu', return_seq=True)

convnet =  tflearn.lstm(convnet, 512, activation='relu', return_seq=True)


convnet =  tflearn.lstm(convnet, 256, activation='relu')

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 512, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 256, activation='relu')
convnet = dropout(convnet, 0.8)

#Input 2 AND 3 PROCESSING processing
convnet1 = input_data(shape=[None, len(array_weights_for_observation)], name='input-2')

convnet1 = tf.reshape(convnet1, shape=[-1, 64, 4])

convnet1 =  tflearn.lstm(convnet1, 1024, activation='relu', return_seq=True)

convnet1 =  tflearn.lstm(convnet1, 512, activation='relu', return_seq=True)

convnet1 =  tflearn.lstm(convnet1, 256, activation='relu')


#convnet2 = input_data(shape=[None, 1], name='input-3')

convnet1 = fully_connected(convnet1, 256, activation='relu')
convnet1 = dropout(convnet, 0.8)

convnet_partial_merge = fully_connected(convnet1, 128, activation='relu')
convnet_partial_merge = dropout(convnet_partial_merge, 0.8)

#convnet2 = fully_connected(convnet2, 128, activation='relu')
#convnet2 = dropout(convnet, 0.8)

#convnet_partial_merge = fully_connected(convnet2, 128, activation='relu')
# MERGE ALL 3 INPUTS FOR FURTHER PROCESSING 

convnet_partial_merge = fully_connected(convnet, 128, activation='relu')
convnet_partial_merge = dropout(convnet_partial_merge, 0.8)


convnet = fully_connected(convnet_partial_merge, 64, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 32, activation='relu')

convnet = fully_connected(convnet, 10, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')



if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')

train_X = X[:-2000]
test_X = X[-2000:]
train_Y = Y[:-2000]
test_Y = Y[-2000:]

X_1 = np.array([i[0] for i in train_X]).reshape(-1,IMG_SIZE,IMG_SIZE,3)
X_2 = np.array([i[1] for i in train_X]).reshape(-1,len(X[0][1]))
#X_3 = np.array([i[2] for i in train_X]).reshape(-1,1)

test_X_1 = np.array([i[0] for i in test_X]).reshape(-1,IMG_SIZE,IMG_SIZE,3)
test_X_2 = np.array([i[1] for i in test_X]).reshape(-1,len(X[0][1]))
#test_X_3 = np.array([i[2] for i in test_X]).reshape(-1,1)

model.fit({'input-1': X_1, 'input-2': X_2}, {'targets': train_Y}, n_epoch=20, validation_set=({'input-1': test_X_1, 'input-2': test_X_2}, {'targets': test_Y}), 
     show_metric=True, run_id=MODEL_NAME, batch_size = 8)

model.save(MODEL_NAME)

Training Step: 89039  | total loss: 1.94768 | time: 177.106s
| Adam | epoch: 020 | loss: 1.94768 - acc: 0.1527 -- iter: 17800/17804
Training Step: 89040  | total loss: 1.94735 | time: 182.542s
| Adam | epoch: 020 | loss: 1.94735 - acc: 0.1499 | val_loss: 1.94616 - val_acc: 0.1410 -- iter: 17804/17804
--
INFO:tensorflow:/home/shekhar/common_agent_9 is not in all_model_checkpoint_paths. Manually adding it.
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'list' object has no attribute 'name'
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'list' object has no attribute 'name'
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'list' object has no attribute 'name'
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'list' object has no attribute 'name'


# TESTING

In [44]:

tf.reset_default_graph()
IMG_SIZE = 100
LR = 1e-3

MODEL_NAME = 'common_agent'


convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 3], name='input-1')

convnet = conv_2d(convnet, 32, 3, activation='relu')
convnet = max_pool_2d(convnet, 3)

convnet = conv_2d(convnet, 64, 3, activation='relu')
convnet = max_pool_2d(convnet, 3)

convnet = conv_2d(convnet, 128, 3, activation='relu')
convnet = max_pool_2d(convnet, 3)
'''
convnet = conv_2d(convnet, 256, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 512, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 256, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 128, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)
'''
convnet = conv_2d(convnet, 64, 3, activation='relu')
convnet = max_pool_2d(convnet, 3)

convnet = conv_2d(convnet, 32, 3, activation='relu')
convnet = max_pool_2d(convnet, 3)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

#Input 2 AND 3 PROCESSING processing
convnet1 = input_data(shape=[None, len(array_weights_for_observation)], name='input-2')

#convnet2 = input_data(shape=[None, 1], name='input-3')

convnet1 = fully_connected(convnet1, 256, activation='relu')
convnet1 = dropout(convnet, 0.8)

convnet_partial_merge = fully_connected(convnet1, 128, activation='relu')

#convnet2 = fully_connected(convnet2, 128, activation='relu')
#convnet2 = dropout(convnet, 0.8)

#convnet_partial_merge = fully_connected(convnet2, 128, activation='relu')
# MERGE ALL 3 INPUTS FOR FURTHER PROCESSING 

convnet_partial_merge = fully_connected(convnet, 128, activation='relu')

convnet = fully_connected(convnet_partial_merge, 64, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 32, activation='relu')

convnet = fully_connected(convnet, 21, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')



if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')
    

env = gym.make("BattleZone-ram-v0")
action = env.action_space.sample()
print(action)
for _ in range(10):
    env.reset()
    score = 0
    prev_obs = []
    while True:
        env.render()
        time.sleep(0.03)
        img = env.render(mode = 'rgb_array')
        img = cv2.resize(img, (100, 100))
        observation, reward, done, info = env.step(action)
        observation = observation.flatten()
        
        if len(observation) > sizeOfArray:     # get all observation array to size of 1000 before saving
            if len(observation) % sizeOfArray == 0:
                observation = np.array(observation)
                observation = observation.reshape(-1,int(observation.size/sizeOfArray)).mean(axis=1)
                observation = np.convolve(np.array(observation),array_weights_for_observation,'same')
                
            else:
                observation = np.array(observation)
                observation = np.append(observation,([0]*(sizeOfArray-(len(observation)%sizeOfArray))))
                observation = observation.reshape(-1,int(observation.size/sizeOfArray)).mean(axis=1)
                observation = np.convolve(np.array(observation),array_weights_for_observation,'same')
                
        else:
            observation = np.array(observation)
            #print(observation.shape)
            #print(array_weights_for_observation.shape)
            observation = np.convolve(observation,array_weights_for_observation,'same')
        
        if prev_obs != []:
            img = img.reshape(-1,IMG_SIZE,IMG_SIZE,3)
            observation_1 = np.array(prev_obs).reshape(-1,len(prev_obs))
            score += reward
            #score_1 = np.array(score).reshape(-1,1)
            action = np.argmax(model.predict([img,observation_1]))
            #print(action)
            action = int(abs(action - random.randint(0,14)))
        prev_obs = list(observation)
        if done:
            print(score)
            break

INFO:tensorflow:Restoring parameters from /home/shekhar/common_agent
model loaded!
16


KeyboardInterrupt: 

In [37]:

tf.reset_default_graph()
IMG_SIZE = 100
LR = 1e-3

MODEL_NAME = 'common_agent_2'


convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 3], name='input-1')

convnet = conv_2d(convnet, 32, 3, activation='relu')
convnet = max_pool_2d(convnet, 3)

convnet = conv_2d(convnet, 64, 3, activation='relu')
convnet = max_pool_2d(convnet, 3)

convnet = conv_2d(convnet, 128, 3, activation='relu')
convnet = max_pool_2d(convnet, 3)
'''
convnet = conv_2d(convnet, 256, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 512, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 256, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 128, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)
'''
convnet = conv_2d(convnet, 64, 3, activation='relu')
convnet = max_pool_2d(convnet, 3)

convnet = conv_2d(convnet, 32, 3, activation='relu')
convnet = max_pool_2d(convnet, 3)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

#Input 2 AND 3 PROCESSING processing
convnet1 = input_data(shape=[None, len(X[0][1])], name='input-2')

#convnet2 = input_data(shape=[None, 1], name='input-3')

convnet1 = fully_connected(convnet1, 512, activation='relu')
convnet1 = dropout(convnet, 0.8)

convnet_partial_merge = fully_connected(convnet1, 256, activation='relu')

#convnet2 = fully_connected(convnet2, 128, activation='relu')
#convnet2 = dropout(convnet, 0.8)

#convnet_partial_merge = fully_connected(convnet2, 128, activation='relu')
# MERGE ALL 3 INPUTS FOR FURTHER PROCESSING 

convnet_partial_merge = fully_connected(convnet, 256, activation='relu')

convnet = fully_connected(convnet_partial_merge, 128, activation='relu')
#convnet = dropout(convnet, 0.8)
convnet = tf.reshape(convnet, shape=[-1, 64, 2])
convnet =  tflearn.lstm(convnet, 256, activation='relu', return_seq=True)

convnet =  tflearn.lstm(convnet, 128, activation='relu')

convnet = fully_connected(convnet, 64, activation='relu')

convnet = fully_connected(convnet, 21, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')



if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')
    

env = gym.make("CartPole-v1")
action = env.action_space.sample()
print(action)
for _ in range(10):
    env.reset()
    score = 0
    prev_obs = []
    while True:
        env.render()
        time.sleep(0.03)
        img = env.render(mode = 'rgb_array')
        img = cv2.resize(img, (100, 100))
        observation, reward, done, info = env.step(action)
        observation = observation.flatten()
        
        if len(observation) > sizeOfArray:     # get all observation array to size of 1000 before saving
            if len(observation) % sizeOfArray == 0:
                observation = np.array(observation)
                observation = observation.reshape(-1,int(observation.size/sizeOfArray)).mean(axis=1)
                observation = np.convolve(np.array(observation),array_weights_for_observation,'same')
                
            else:
                observation = np.array(observation)
                observation = np.append(observation,([0]*(sizeOfArray-(len(observation)%sizeOfArray))))
                observation = observation.reshape(-1,int(observation.size/sizeOfArray)).mean(axis=1)
                observation = np.convolve(np.array(observation),array_weights_for_observation,'same')
                
        else:
            observation = np.array(observation)
            #print(observation.shape)
            #print(array_weights_for_observation.shape)
            observation = np.convolve(observation,array_weights_for_observation,'same')
        
        if prev_obs != []:
            img = img.reshape(-1,IMG_SIZE,IMG_SIZE,3)
            observation_1 = np.array(prev_obs).reshape(-1,len(prev_obs))
            score += reward
            #score_1 = np.array(score).reshape(-1,1)
            action = np.argmax(model.predict([img,observation_1]))
            action = int(abs(action - random.randint(0,6)))

        prev_obs = list(observation)
        if done:
            print(score)
            break

INFO:tensorflow:Restoring parameters from /home/shekhar/common_agent_2


InvalidArgumentError: Assign requires shapes of both tensors to match. lhs shape= [256,512] rhs shape= [1000,512]
	 [[Node: save_1/Assign_40 = Assign[T=DT_FLOAT, _class=["loc:@FullyConnected_1/W"], use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](FullyConnected_1/W, save_1/RestoreV2/_147)]]
	 [[Node: save_1/RestoreV2/_94 = _Send[T=DT_FLOAT, client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device_incarnation=1, tensor_name="edge_96_save_1/RestoreV2", _device="/job:localhost/replica:0/task:0/device:CPU:0"](save_1/RestoreV2:7)]]

Caused by op u'save_1/Assign_40', defined at:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py", li

In [36]:
# common agent 3

tf.reset_default_graph()
IMG_SIZE = 100
LR = 1e-3

MODEL_NAME = 'common_agent_3'


convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 3], name='input-1')

convnet = conv_2d(convnet, 32, 3, activation='relu')
convnet = max_pool_2d(convnet, 3)

convnet = conv_2d(convnet, 64, 3, activation='relu')
convnet = max_pool_2d(convnet, 3)

convnet = conv_2d(convnet, 128, 3, activation='relu')
convnet = max_pool_2d(convnet, 3)
'''
convnet = conv_2d(convnet, 256, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 512, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 256, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 128, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)
'''
convnet = conv_2d(convnet, 64, 3, activation='relu')
convnet = max_pool_2d(convnet, 3)

convnet = conv_2d(convnet, 32, 3, activation='relu')
convnet = max_pool_2d(convnet, 3)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 512, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 256, activation='relu')
convnet = dropout(convnet, 0.8)

#Input 2 AND 3 PROCESSING processing
convnet1 = input_data(shape=[None, len(array_weights_for_observation)], name='input-2')

#convnet2 = input_data(shape=[None, 1], name='input-3')

convnet1 = fully_connected(convnet1, 256, activation='relu')
convnet1 = dropout(convnet, 0.8)

convnet_partial_merge = fully_connected(convnet1, 128, activation='relu')

#convnet2 = fully_connected(convnet2, 128, activation='relu')
#convnet2 = dropout(convnet, 0.8)

#convnet_partial_merge = fully_connected(convnet2, 128, activation='relu')
# MERGE ALL 3 INPUTS FOR FURTHER PROCESSING 

convnet_partial_merge = fully_connected(convnet, 128, activation='relu')

convnet = fully_connected(convnet_partial_merge, 64, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 32, activation='relu')

convnet = fully_connected(convnet, 21, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')



if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')
    
env = gym.make(env_name)
action = env.action_space.sample()
print(action)
for _ in range(10):
    env.reset()
    score = 0
    prev_obs = []
    while True:
        env.render()
        time.sleep(0.03)
        img = env.render(mode = 'rgb_array')
        img = cv2.resize(img, (100, 100))
        observation, reward, done, info = env.step(action)
        observation = observation.flatten()
        
        if len(observation) > sizeOfArray:     # get all observation array to size of 1000 before saving
            if len(observation) % sizeOfArray == 0:
                observation = np.array(observation)
                observation = observation.reshape(-1,int(observation.size/sizeOfArray)).mean(axis=1)
                observation = np.convolve(np.array(observation),array_weights_for_observation,'same')
                
            else:
                observation = np.array(observation)
                observation = np.append(observation,([0]*(sizeOfArray-(len(observation)%sizeOfArray))))
                observation = observation.reshape(-1,int(observation.size/sizeOfArray)).mean(axis=1)
                observation = np.convolve(np.array(observation),array_weights_for_observation,'same')
                
        else:
            observation = np.array(observation)
            #print(observation.shape)
            #print(array_weights_for_observation.shape)
            observation = np.convolve(observation,array_weights_for_observation,'same')
        
        if prev_obs != []:
            img = img.reshape(-1,IMG_SIZE,IMG_SIZE,3)
            observation_1 = np.array(prev_obs).reshape(-1,len(prev_obs))
            score += reward
            #score_1 = np.array(score).reshape(-1,1)
            action = np.argmax(model.predict([img,observation_1]))
        prev_obs = list(observation)
        if done:
            print(score)
            break

INFO:tensorflow:Restoring parameters from /home/shekhar/common_agent_3
model loaded!
WARN: gym.spaces.Box autodetected dtype as <type 'numpy.float32'>. Please provide explicit dtype.
0
12.0
8.0
8.0
8.0
9.0
9.0
9.0
8.0
9.0
7.0


In [7]:
tf.reset_default_graph()
IMG_SIZE = 100
LR = 1e-3

MODEL_NAME = 'common_agent_5'


convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 3], name='input-1')

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 128, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 512, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 256, activation='relu')
convnet = dropout(convnet, 0.8)

#Input 2 AND 3 PROCESSING processing
convnet1 = input_data(shape=[None, len(array_weights_for_observation)], name='input-2')

#convnet2 = input_data(shape=[None, 1], name='input-3')

convnet1 = fully_connected(convnet1, 256, activation='relu')
convnet1 = dropout(convnet, 0.8)

convnet_partial_merge = fully_connected(convnet1, 128, activation='relu')

#convnet2 = fully_connected(convnet2, 128, activation='relu')
#convnet2 = dropout(convnet, 0.8)

#convnet_partial_merge = fully_connected(convnet2, 128, activation='relu')
# MERGE ALL 3 INPUTS FOR FURTHER PROCESSING 

convnet_partial_merge = fully_connected(convnet, 128, activation='relu')

convnet = fully_connected(convnet_partial_merge, 64, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 32, activation='relu')

convnet = fully_connected(convnet, 21, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')



if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')

env = gym.make(env_name)
action = env.action_space.sample()
print(action)
for _ in range(10):
    env.reset()
    score = 0
    prev_obs = []
    while True:
        env.render()
        time.sleep(0.03)
        img = env.render(mode = 'rgb_array')
        img = cv2.resize(img, (100, 100))
        observation, reward, done, info = env.step(action)
        observation = observation.flatten()
        
        if len(observation) > sizeOfArray:     # get all observation array to size of 1000 before saving
            if len(observation) % sizeOfArray == 0:
                observation = np.array(observation)
                observation = observation.reshape(-1,int(observation.size/sizeOfArray)).mean(axis=1)
                observation = np.convolve(np.array(observation),array_weights_for_observation,'same')
                
            else:
                observation = np.array(observation)
                observation = np.append(observation,([0]*(sizeOfArray-(len(observation)%sizeOfArray))))
                observation = observation.reshape(-1,int(observation.size/sizeOfArray)).mean(axis=1)
                observation = np.convolve(np.array(observation),array_weights_for_observation,'same')
                
        else:
            observation = np.array(observation)
            #print(observation.shape)
            #print(array_weights_for_observation.shape)
            observation = np.convolve(observation,array_weights_for_observation,'same')
        
        if prev_obs != []:
            img = img.reshape(-1,IMG_SIZE,IMG_SIZE,3)
            observation_1 = np.array(prev_obs).reshape(-1,len(prev_obs))
            score += reward
            #score_1 = np.array(score).reshape(-1,1)
            action = np.argmax(model.predict([img,observation_1]))
        prev_obs = list(observation)
        if done:
            print(score)
            break


INFO:tensorflow:Restoring parameters from /home/shekhar/common_agent_5
model loaded!


/home/shekhar/gym/gym/__init__.py:22: UserWarning: DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.
  warnings.warn('DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.')


12
3000.0


KeyboardInterrupt: 

In [35]:
tf.reset_default_graph()
IMG_SIZE = 100
LR = 1e-3

MODEL_NAME = 'common_agent_7'


convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 3], name='input-1')

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 128, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 256, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 128, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 512, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 256, activation='relu')
convnet = dropout(convnet, 0.8)

#Input 2 AND 3 PROCESSING processing
convnet1 = input_data(shape=[None, len(array_weights_for_observation)], name='input-2')

#convnet2 = input_data(shape=[None, 1], name='input-3')

convnet1 = fully_connected(convnet1, 256, activation='relu')
convnet1 = dropout(convnet, 0.8)

convnet_partial_merge = fully_connected(convnet1, 128, activation='relu')
convnet_partial_merge = dropout(convnet_partial_merge, 0.8)

#convnet2 = fully_connected(convnet2, 128, activation='relu')
#convnet2 = dropout(convnet, 0.8)

#convnet_partial_merge = fully_connected(convnet2, 128, activation='relu')
# MERGE ALL 3 INPUTS FOR FURTHER PROCESSING 

convnet_partial_merge = fully_connected(convnet, 128, activation='relu')
convnet_partial_merge = dropout(convnet_partial_merge, 0.8)


convnet = fully_connected(convnet_partial_merge, 64, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 32, activation='relu')

convnet = fully_connected(convnet, 10, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')



if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')

    
env = gym.make(env_name)
action = env.action_space.sample()
print(action)
for _ in range(10):
    env.reset()
    score = 0
    prev_obs = []
    while True:
        env.render()
        time.sleep(0.03)
        img = env.render(mode = 'rgb_array')
        img = cv2.resize(img, (100, 100))
        observation, reward, done, info = env.step(action)
        observation = observation.flatten()
        
        if len(observation) > sizeOfArray:     # get all observation array to size of 1000 before saving
            if len(observation) % sizeOfArray == 0:
                observation = np.array(observation)
                observation = observation.reshape(-1,int(observation.size/sizeOfArray)).mean(axis=1)
                observation = np.convolve(np.array(observation),array_weights_for_observation,'same')
                
            else:
                observation = np.array(observation)
                observation = np.append(observation,([0]*(sizeOfArray-(len(observation)%sizeOfArray))))
                observation = observation.reshape(-1,int(observation.size/sizeOfArray)).mean(axis=1)
                observation = np.convolve(np.array(observation),array_weights_for_observation,'same')
                
        else:
            observation = np.array(observation)
            #print(observation.shape)
            #print(array_weights_for_observation.shape)
            observation = np.convolve(observation,array_weights_for_observation,'same')
        
        if prev_obs != []:
            img = img.reshape(-1,IMG_SIZE,IMG_SIZE,3)
            observation_1 = np.array(prev_obs).reshape(-1,len(prev_obs))
            score += reward
            #score_1 = np.array(score).reshape(-1,1)
            action = np.argmax(model.predict([img,observation_1]))
            print(action)
            action = int(abs(action - random.randint(0,6)))

        prev_obs = list(observation)
        if done:
            print(score)
            break


INFO:tensorflow:Restoring parameters from /home/shekhar/common_agent_7
model loaded!
4
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0

KeyboardInterrupt: 

In [30]:

tf.reset_default_graph()
IMG_SIZE = 100
LR = 1e-3

MODEL_NAME = 'common_agent_8'


convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 3], name='input-1')

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 128, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 256, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 128, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = tf.reshape(convnet, shape=[-1, 16, 2])
convnet =  tflearn.lstm(convnet, 128, activation='relu', return_seq=True)

convnet =  tflearn.lstm(convnet, 128, activation='relu')

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 512, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 256, activation='relu')
convnet = dropout(convnet, 0.8)

#Input 2 AND 3 PROCESSING processing
convnet1 = input_data(shape=[None, len(array_weights_for_observation)], name='input-2')

#convnet2 = input_data(shape=[None, 1], name='input-3')

convnet1 = fully_connected(convnet1, 256, activation='relu')
convnet1 = dropout(convnet, 0.8)

convnet_partial_merge = fully_connected(convnet1, 128, activation='relu')
convnet_partial_merge = dropout(convnet_partial_merge, 0.8)

#convnet2 = fully_connected(convnet2, 128, activation='relu')
#convnet2 = dropout(convnet, 0.8)

#convnet_partial_merge = fully_connected(convnet2, 128, activation='relu')
# MERGE ALL 3 INPUTS FOR FURTHER PROCESSING 

convnet_partial_merge = fully_connected(convnet, 128, activation='relu')
convnet_partial_merge = dropout(convnet_partial_merge, 0.8)


convnet = fully_connected(convnet_partial_merge, 64, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 32, activation='relu')

convnet = fully_connected(convnet, 10, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')



if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')

env = gym.make(env_name)
action = env.action_space.sample()
print(action)
for _ in range(10):
    env.reset()
    score = 0
    prev_obs = []
    while True:
        env.render()
        time.sleep(0.03)
        img = env.render(mode = 'rgb_array')
        img = cv2.resize(img, (100, 100))
        observation, reward, done, info = env.step(action)
        observation = observation.flatten()
        
        if len(observation) > sizeOfArray:     # get all observation array to size of 1000 before saving
            if len(observation) % sizeOfArray == 0:
                observation = np.array(observation)
                observation = observation.reshape(-1,int(observation.size/sizeOfArray)).mean(axis=1)
                observation = np.convolve(np.array(observation),array_weights_for_observation,'same')
                
            else:
                observation = np.array(observation)
                observation = np.append(observation,([0]*(sizeOfArray-(len(observation)%sizeOfArray))))
                observation = observation.reshape(-1,int(observation.size/sizeOfArray)).mean(axis=1)
                observation = np.convolve(np.array(observation),array_weights_for_observation,'same')
                
        else:
            observation = np.array(observation)
            #print(observation.shape)
            #print(array_weights_for_observation.shape)
            observation = np.convolve(observation,array_weights_for_observation,'same')
        
        if prev_obs != []:
            img = img.reshape(-1,IMG_SIZE,IMG_SIZE,3)
            observation_1 = np.array(prev_obs).reshape(-1,len(prev_obs))
            score += reward
            #score_1 = np.array(score).reshape(-1,1)
            action = np.argmax(model.predict([img,observation_1]))
            action = int(action - random.randint(0,10))
        prev_obs = list(observation)
        if done:
            print(score)
            break


INFO:tensorflow:Restoring parameters from /home/shekhar/common_agent_8
model loaded!
5
189.0
315.0
189.0
441.0
273.0
189.0
189.0
147.0
252.0
378.0
